In [3]:
!pip install gsutil --upgrade --quiet

In [4]:
import shutil

shutil.rmtree("/kaggle/working/trocr_frozen_head/")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/trocr_frozen_head/'

In [2]:
!mkdir -p /kaggle/working/mydata
## !rm -rf /kaggle/working/*



In [5]:
!pip install -U transformers

In [4]:
!gsutil -m cp -r gs://adoocr-data/train /kaggle/working/mydata/
!gsutil -m cp -r gs://adoocr-data/test  /kaggle/working/mydata/ 

Copying gs://adoocr-data/train/X_trainp_pg_vg.npy...
Copying gs://adoocr-data/train/mapping_labls...
Copying gs://adoocr-data/train/y_trainp_pg_vg.npy...                            
Copying gs://adoocr-data/test/X_test_vg.npy...84.1 MiB/s ETA 00:00:00           
Copying gs://adoocr-data/test/mapping_labls...                                  
Copying gs://adoocr-data/test/x_test_pg.npy...                                  
Copying gs://adoocr-data/test/x_testp.npy...
Copying gs://adoocr-data/test/y_test_pg.npy...                                  
Copying gs://adoocr-data/test/y_test_vg.npy...                                  
Copying gs://adoocr-data/test/y_testp.npy...


In [11]:
# %%bash
# cat <<EOF > /kaggle/working/kaggle.json
# {"username":"abiywondimu","key":"31fd71809a2ceeb88a6e435c42bfed37"}
# EOF

# # secure the file
# chmod 600 /kaggle/working/kaggle.json


In [12]:
# %%bash
# mkdir -p ~/.kaggle
# mv /kaggle/working/kaggle.json ~/.kaggle/


In [13]:
# !cd /kaggle/working/mydata && kaggle datasets init

In [14]:
# !cd /kaggle/working/mydata
# import json

# meta_path = "/kaggle/working/mydata/dataset-metadata.json"
# with open(meta_path, "r", encoding="utf-8") as f:
#     data = json.load(f)

# # >>> edit your fields:
# data["title"] = "my amharic ocr lines"
# data["id"]    = "abiywondimu/amharic-textline-ocr"
# data["isPrivate"] = True

# with open(meta_path, "w", encoding="utf-8") as f:
#     json.dump(data, f, ensure_ascii=False, indent=2)
# print("Re‐wrote metadata!")


In [15]:
# !cd /kaggle/working/mydata

# !kaggle datasets create -p /kaggle/working/mydata --dir-mode tar

In [16]:
# !kaggle config view

In [6]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    TrainingArguments,
    Trainer,
    default_data_collator
)
import editdistance

2025-06-26 11:48:24.496908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750938504.705853    1219 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750938504.770134    1219 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
MODEL_NAME = "microsoft/trocr-small-printed"
BATCH_SIZE = 32            
EPOCHS     = 3
LR         = 5e-5
OUTPUT_DIR = "./trocr_frozen_head"

In [8]:
# ─── 2) DATASET ───────────────────────────────────────────────────────────────
class LineOCRDataset(Dataset):
    def __init__(self, images_np, labels_np, processor, idx2char, max_length=64):
        self.X = images_np        # shape (N,48,128), dtype uint8 or float32
        self.Y = labels_np        # shape (N, max_label_len), padded with 0
        self.proc = processor
        self.idx2char = idx2char
        self.max_length = max_length

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        img = self.X[i].astype(np.uint8)
        # TrOCRProcessor expects PIL or np.array H×W×3, so stack channels:
        img3 = np.stack([img, img, img], axis=-1)
        pixel_values = self.proc(images=img3, return_tensors="pt").pixel_values[0]

        # decode Y[i] to string
        seq = self.Y[i]
        chars = []
        for idx in seq:
            if idx == 0: break  # pad
            chars.append(self.idx2char[int(idx)])
        text = "".join(chars)

        labels = self.proc.tokenizer(
            text, padding="max_length", truncation=True,
            max_length=self.max_length, return_tensors="pt"
        ).input_ids[0]

        return {
            "pixel_values": pixel_values,
            "labels": labels,
        }

In [9]:
# ─── 3) METRICS ───────────────────────────────────────────────────────────────
def compute_metrics(pred):
    pred_ids = pred.predictions.argmax(-1) if pred.predictions.ndim==3 else pred.predictions
    pred_texts = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids  = pred.label_ids
    label_ids[label_ids==-100] = processor.tokenizer.pad_token_id
    gt_texts   = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    total_cer = total_wer = total_chars = total_words = correct_chars = 0
    for p, g in zip(pred_texts, gt_texts):
        # CER
        total_cer  += editdistance.eval(p, g)
        total_chars+= len(g)
        # WER
        pw = p.split(); gw = g.split()
        total_wer  += editdistance.eval(pw, gw)
        total_words+= len(gw)
        # Char acc
        for pc, gc in zip(p, g):
            if pc==gc: correct_chars+=1

    return {
        "cer": total_cer/total_chars,
        "wer": total_wer/total_words,
        "char_acc": correct_chars/total_chars,
    }

In [10]:
# ─── 4) LOAD DATA ─────────────────────────────────────────────────────────────
# your .npy paths
X_train = np.load("/kaggle/working/mydata/train/X_trainp_pg_vg.npy", mmap_mode="r")[:80000]
Y_train = np.load("/kaggle/working/mydata/train/y_trainp_pg_vg.npy", allow_pickle=True)[:80000]
X_val   = np.load("/kaggle/working/mydata/test/x_testp.npy",  mmap_mode="r")
Y_val   = np.load("/kaggle/working/mydata/test/y_testp.npy",   allow_pickle=True)

# build idx2char from your mapping file
idx2char = {}

special_tokens = {
    "<space>": " ",
    "<newline>": "\n",
    "<tab>": "\t"
}

with open("/kaggle/working/mydata/train/mapping_labls", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            i, ch = parts
            ch = special_tokens.get(ch, ch)  # convert <space> etc to actual char
            idx2char[int(i)] = ch



processor = TrOCRProcessor.from_pretrained(MODEL_NAME)

train_ds = LineOCRDataset(X_train, Y_train, processor, idx2char)
val_ds   = LineOCRDataset(X_val,   Y_val,   processor, idx2char)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [11]:
# print("Sample label indices:", dataset[0]["label_ids"])

print("Available label indices:", sorted(idx2char.keys()))


Available label indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,

In [12]:
# ─── 5) MODEL ────────────────────────────────────────────────────────────────
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)

# freeze entire encoder
for param in model.encoder.parameters():
    param.requires_grad = False

# set pad token
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id           = processor.tokenizer.pad_token_id
model.config.vocab_size             = model.config.decoder.vocab_size

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import EarlyStoppingCallback

# ─── 6) TRAINER ──────────────────────────────────────────────────────────────
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    save_total_limit=2,
    fp16=True,
    learning_rate=LR,
    logging_steps=200,
    report_to="none",
    save_strategy="no",  # no save if you're not evaluating
    eval_strategy="no"  # disable eval
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    # eval_dataset=val_ds,  # <-- comment or remove this line
    data_collator=default_data_collator,
    compute_metrics=None,
    callbacks=[]
)



In [15]:
# ─── 7) TRAIN ─────────────────────────────────────────────────────────────────
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,0.643400
400,0.380900
600,0.372700
800,0.366100
1000,0.363200
1200,0.362000
1400,0.359800
1600,0.356700
1800,0.354000
2000,0.352600


TrainOutput(global_step=3750, training_loss=0.3704139149983724, metrics={'train_runtime': 3982.4452, 'train_samples_per_second': 60.264, 'train_steps_per_second': 0.942, 'total_flos': 2.870992498065408e+19, 'train_loss': 0.3704139149983724, 'epoch': 3.0})

In [16]:
# ─── 8) SAVE ──────────────────────────────────────────────────────────────────
trainer.save_model(OUTPUT_DIR)

In [17]:
!zip -r /kaggle/working/trocr_frozen_head.zip /kaggle/working/trocr_frozen_head

  adding: kaggle/working/trocr_frozen_head/ (stored 0%)
  adding: kaggle/working/trocr_frozen_head/training_args.bin (deflated 52%)
  adding: kaggle/working/trocr_frozen_head/config.json (deflated 65%)
  adding: kaggle/working/trocr_frozen_head/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 28%)
  adding: kaggle/working/trocr_frozen_head/generation_config.json (deflated 35%)


In [27]:
!pip install -q google-cloud-storage

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 4.1 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.25.1 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [28]:
!gsutil -m cp -r gs://adoocr-data/train/service-account-key.json /kaggle/working/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Copying gs://adoocr-data/train/service-account-key.json...
/ [1/1 files][  2.3 KiB/  2.3 KiB] 100% Done                                    
Operation completed over 1 objects/2.3 KiB.                                      


In [34]:
from google.cloud import storage
import os

# Path to service account key
key_path = "/kaggle/working/service-account-key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path

# Initialize GCS client
client = storage.Client()
bucket_name = 'adoocr-data'  # TODO: Replace this with your actual bucket name
destination_blob_name = 'trocr_frozen_head.zip'
source_file_name = '/kaggle/working/trocr_frozen_head.zip'

# Upload file
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

print("✅ Upload to GCP completed!")
print(f"🔗 File URL: https://storage.googleapis.com/{bucket_name}/{destination_blob_name}")


✅ Upload to GCP completed!
🔗 File URL: https://storage.googleapis.com/adoocr-data/trocr_frozen_head.zip


In [33]:
for blob in bucket.list_blobs():
    print(blob.name)

NameError: name 'bucket' is not defined